In [ ]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
import matplotlib.pyplot as plt

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

In [ ]:
trainset = datasets.MNIST(r'..\input\MNIST', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=50, shuffle=True)

testset = datasets.MNIST(r'..\input\MNIST', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=50, shuffle=True)

In [ ]:
class Residual_Block(nn.Module):
    def __init__(self, dim):
        super(Residual_Block,self).__init__()
        # Residual Block
        self.residual_block = nn.Sequential(
                nn.Conv2d(dim, dim, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.Conv2d(dim, dim, kernel_size=3, padding=1),
            )            
        self.relu = nn.ReLU()
                  
    def forward(self, x):
        out = self. residual_block(x)  # F(x)
        out = out + x  # F(x) + x
        out = self.relu(out)
        return out

In [ ]:
class model3(nn.Module):
    def __init__(self):
        super(model3, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.residual = Residual_Block(32)
        self.after =  nn.Sequential(nn.MaxPool2d(kernel_size=2),
                    nn.Flatten(),
                    nn.Linear(6272,64), 
                    nn.ReLU(),
                    nn.Linear(64,10),
                    nn.LogSoftmax(dim=1))
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.residual(x)
        x = self.residual(x)
        output = self.after(x)
        
        return output

In [ ]:
model1=nn.Sequential(nn.Conv2d(in_channels=1,out_channels=32,stride=1,padding=1,kernel_size=3),
                    nn.ReLU(),          
                    nn.Conv2d(in_channels=32,out_channels=64,stride=1,padding=1,kernel_size=3), 
                    nn.ReLU(),          
                    nn.MaxPool2d(kernel_size=2),
                    nn.Flatten(),
                    nn.Linear(12544,64), 
                    nn.ReLU(),
                    nn.Linear(64,10),
                    nn.LogSoftmax(dim=1) 

                  ) 

print(model1)

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Flatten(start_dim=1, end_dim=-1)
  (6): Linear(in_features=12544, out_features=64, bias=True)
  (7): ReLU()
  (8): Linear(in_features=64, out_features=10, bias=True)
  (9): LogSoftmax(dim=1)
)


In [ ]:
model2=nn.Sequential(nn.Conv2d(in_channels=1,out_channels=8,stride=1,padding=1,kernel_size=3),
                    nn.ReLU(),          
                    nn.Conv2d(in_channels=8,out_channels=16,stride=1,padding=1,kernel_size=3), 
                    nn.ReLU(),          
                    nn.MaxPool2d(kernel_size=2),
                    nn.Conv2d(in_channels=16,out_channels=32,stride=1,padding=1,kernel_size=3), 
                    nn.ReLU(),
                    nn.Conv2d(in_channels=32,out_channels=64,stride=1,padding=1,kernel_size=3), 
                    nn.ReLU(),
                    nn.MaxPool2d(kernel_size=2),
                    nn.Flatten(),
                    nn.Linear(64*7*7,64), 
                    nn.ReLU(),
                    nn.Linear(64,10),
                    nn.LogSoftmax(dim=1) 

                  ) 



In [ ]:
def test(model,test_loader):
    model.eval()
    correct = 0
    for data, target in test_loader:
        with torch.no_grad():
            output = model(data)    
        prediction = output.data.max(1)[1]
        correct += prediction.eq(target.data).sum()
        
    print('\nTest set: Accuracy: {:.2f}%'.format(100. * correct / len(test_loader.dataset)))
    return (correct / len(test_loader.dataset)).item()

In [ ]:
def plot(loss,acc):

    plt.xlabel("epoch")
    plt.ylabel("loss")
    plt.plot(loss)
    plt.show()
    plt.xlabel("epoch")
    plt.ylabel("accuracy")
    plt.plot(acc)
    plt.axis([0, 30, 0.8, 1])
    plt.show()



In [ ]:
def train(num,model,trainloader,testloader):   
    epochs = 30 # total number of iteration for training
    running_loss_list= []
    test_acc_list =[]
    epochs_list = []
    optimizer = optim.Adam(model.parameters())
    criterion = nn.NLLLoss()

    for e in range(epochs):
        running_loss = 0
        for images, labels in trainloader:

            # defining gradient in each epoch as 0
            optimizer.zero_grad()
            
            # modeling for each image batch
            output = model(images)
            
            # calculating the loss
            loss = criterion(output, labels)
            
            # This is where the model learns by backpropagating
            loss.backward()
            
            # And optimizes its weights here
            optimizer.step()
            
            # calculating the loss
            running_loss += loss.item()
            
        else:
            print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))
            
            running_loss_list.append(running_loss/len(trainloader))
            test_acc_list.append(test(model,testloader))



    # Save model
    torch.save(model, 'model'+ num +'.pt')
    print(running_loss_list)
    print(test_acc_list)
    return running_loss_list, test_acc_list

In [ ]:
loss, acc = train("1",model1,trainloader,testloader)

plot(loss,acc)

In [ ]:
loss, acc = train("2",model2,trainloader,testloader)

plot(loss,acc)

In [ ]:
loss, acc = train("3",model3(),trainloader,testloader)

plot(loss,acc)